## Installing Required Libraries

The Jupyter environment does not come pre-installed with the necessary library below. Nevertheless, if you are using this notebook in an alternate Jupyter environment, you will be required to install these library. To do so, simply uncomment the code cell below by removing the '#' sign before '!pip'.

In [ ]:
!pip install prophet
# Note: If your environment doesn't support "!mamba install", use "!pip install"

## Importing Required Libraries

In [1]:
import pandas as pd
from prophet import Prophet
from matplotlib import pyplot
from matplotlib.pyplot import figure
from sklearn.metrics import mean_absolute_error
import plotly.express as px
import plotly.graph_objects as go

## Read .csv File

Read a comma-seperated values (csv) file into DataFrame.

This code disables SSL certificate validation for HTTPS requests, allowing you to proceed without certificate checks. Remember that this approach should be used sparingly and only when you are confident about the safety and trustworthiness of the data source. In production environments, it's essential to prioritize security and maintain proper SSL certificate validation.

In [ ]:
#To bypass SSL verification in Jupyter Notebook script
import ssl
ssl._create_default_https_context = ssl._create_unverified_context